<style>
    h1 {font-family:Times; font-size:20pt}
    h2 {font-family:Times; font-size:18pt}
    h3 {font-family:Times; font-size:16pt}
    p {font-family:Times; font-size:12pt}
    </style>
    
> # Earth Satellites
>
> Skyfield is able to predict the positions of Earth satellites by loading satellite orbital elements from Two-Line Element (TLE) files — published by organizations like CelesTrak — and running them through the SGP4 satellite propagation routine. 

## TLE file format:
>AAAAAAAAAAAAAAAAAAAAAAAA

>1 NNNNNU NNNNNAAA NNNNN.NNNNNNNN +.NNNNNNNN +NNNNN-N +NNNNN-N N NNNNN

>2 NNNNN NNN.NNNN NNN.NNNN NNNNNNN NNN.NNNN NNN.NNNN NN.NNNNNNNNNNNNNN

- Line 0:  corresponds to the Satelletie catalog number (NORAD SATCAT) = (the name of the satellite)
- Line 1:

Reference : https://celestrak.com/NORAD/documentation/tle-fmt.php

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import interpolate
import os

from skyfield.api import load, wgs84

<style>
    h1 {font-family:Times; font-size:20pt}
    h2 {font-family:Times; font-size:18pt}
    h3 {font-family:Times; font-size:16pt}
    p {font-family:Times; font-size:12pt}
    </style>
    
> ## Loading a TLE file
>
> Skyfield loader objects offer a tle_file() method that can download and cache a file full of satellite elements from a site like Celestrak. A popular observing target for satellite observers is the International Space Station, which is listed in Celestrak’s stations.txt file:

In [ ]:
# stations_url = 'https://www.celestrak.com/NORAD/elements/geo.txt' # Retrieve geo satellite from celestrak
stations_url = 'https://celestrak.com/NORAD/elements/weather.txt' # Retrieve weather satellites


satellites = load.tle_file(stations_url, reload=False)

by_name = {sat.name: sat for sat in satellites}
satellite = by_name['METOP-B']

print(satellite)

<style>
    h1 {font-family:Times; font-size:20pt}
    h2 {font-family:Times; font-size:18pt}
    h3 {font-family:Times; font-size:16pt}
    p {font-family:Times; font-size:12pt}
    </style>
    
> ## Loading a TLE set from strings
> If your program already has the two lines of TLE data for a satellite and doesn’t need Skyfield to download and parse a Celestrak file, you can instantiate an EarthSatellite directly.

In [ ]:
from skyfield.api import EarthSatellite

ts = load.timescale()
#line1 = '1 38771U 12049A   21328.15653177  .00000019  00000+0  28851-4 0  9990'
#line2 = '2 38771  98.6954  26.0543 0000110 196.3188 189.2859 14.21494351476561'
#line1 = '1 38771U 12049A   21345.19892716  .00000005  00000+0  22412-4 0  9999'
#line2 = '2 38771  98.6932  42.8377 0000999 161.9725 266.8848 14.21496788478982'

#satellite = EarthSatellite(line1, line2, 'METOP-A', ts)

#print(ts)
#print(satellite)

<style>
    h1 {font-family:Times; font-size:20pt}
    h2 {font-family:Times; font-size:18pt}
    h3 {font-family:Times; font-size:16pt}
    p {font-family:Times; font-size:12pt}
    </style>
>
> ## Satellite altitude, azimuth, and distance
>
> You might be most interested in whether the satellite is above or below the horizon from your own position as an observer, and in which direction to look for it. If you build an object to represent your latitude and longitude , you can use vector subtraction to ask “where will the satellite be relative to my location?”

In [ ]:

Lavrion = wgs84.latlon(37.72, 24.048)
Ljubljana = wgs84.latlon(46.056, 14.506)
difference = satellite - Ljubljana

dates = pd.date_range("2020-01-01","2021-01-1",freq='60S')
alt = np.zeros(np.size(dates)) * np.nan
az = np.zeros(np.size(dates)) * np.nan
distances = np.zeros(np.size(dates)) * np.nan

print(dates.shape)
count = 0

city_name = "lavrion" ## Edit here to save in another file ljubljana ou lavrion
pkl_name= f"./coordinates_{city_name}.pkl" 
pkl_pdf_name= f"./stat_{city_name}.pkl" 
pkl_available = True

try:
    coord_df = pd.DataFrame(pd.read_pickle(pkl_name))
except:
    pkl_available=False

print(pkl_available)
if (pkl_available==False):
    for ii,date in enumerate(dates):
        count+=1
        if (count%100000==0) :
            print(count)
        t = ts.utc(date.year, date.month, date.day, date.hour, date.minute, date.second)

        topocentric = difference.at(t)

        alt_, az_, distance = topocentric.altaz()
        
        distances[ii] = distance.km
        alt[ii] = alt_.degrees
        az[ii] = az_.degrees

    coord_df = pd.DataFrame(index = dates)

    # ind = np.nonzero(alt<5)

    # alt[ind] = np.nan
    # az[ind] = np.nan 

    coord_df['EL'] = alt 
    coord_df['AZ'] = az 
    coord_df['DISTANCES'] = distances

    coord_df.to_pickle(pkl_name)



In [ ]:
print("Plotting")
Plot = True
if Plot:
    fig, ax = plt.subplots(dpi=200)
    ax = coord_df.plot(ax=ax, y='EL', linestyle='solid', marker='')

    ax.grid(b=True, which='major', axis='both')
    ax.axhline(y=5, color='r', linestyle='-')

    plt.show()

<style>
    h1 {font-family:Times; font-size:20pt}
    h2 {font-family:Times; font-size:18pt}
    h3 {font-family:Times; font-size:16pt}
    p {font-family:Times; font-size:12pt}
    </style>
>
> ## Probability density function
>
> In this section the probability density function for elevation angle will be calculated $P(\theta)$

In [ ]:
stp = 5
bins = np.arange(5,91,stp) # Regroup in block of 5°, threshold 
stat = pd.DataFrame(index=bins)

for bin in bins:
    ind  = np.logical_and( bin <= coord_df['EL'] , coord_df['EL'] < bin+stp) # For all elevation that are in the range bin, bin+stp

    el = coord_df.loc[ind,'EL'] # Select all elevation that are in the current bin, bin+stp range

    stat.loc[bin,'PDF'] = np.size(el)/np.size(ind) * 100 # Probability of presensce in the current bin: PDF = (#number of element in the current bin)/(#number total number of element)

# Save pdf to pickle
stat.to_pickle(pkl_pdf_name)


fig, ax = plt.subplots(dpi=200)
ax = stat.plot(ax=ax, y='PDF', kind='bar')
ax.grid(b=True, which='major', axis='y')

ax.set_xlabel(r'Elevation [$^\circ$]')
ax.set_ylabel(r'Probability [%]')
plt.show()



In [ ]:
# Load picke pdf, if pdf was already computed
stat = pd.read_pickle(pkl_pdf_name)
stat['PDF'] = stat['PDF']/stat['PDF'].sum() * 100

fig, ax = plt.subplots(dpi=200)
ax = stat.plot(ax=ax, y='PDF', kind='bar')
ax.grid(b=True, which='major', axis='y')

ax.set_xlabel(r'Elevation [$^\circ$]')
ax.set_ylabel(r'Probability [%]')
plt.show()


# Compute average of attenuation
## Loading data 

In [ ]:

folders_name = ["Lavrion-37.5GHz-5-50", "Lavrion-37.5GHz-55-89.9", "Lavrion-75GHz-5-50", "Lavrion-75-GHz-55-89.9", "Ljubljana-37.5GHz-5-50", "Ljubljana-37.5GHZ-55-89.9", "Ljubljana-75GHz-5-50", "Ljubljana-75GHz-55-89.9"]

attenuations_file = {'total': "attenuation_total.csv", "cloud": "attenuation_cloud.csv", "oxygen":"attenuation_oxygen.csv", "rain":"attenuation_rain.csv", "scintillation":"attenuation_scintillation.csv"}
att_columns = {'total':'att_comb_dB', 'cloud':'att_cld_dB', 'oxygen':'', 'rain':'att_rain_dB', 'scintillation':'att_scin_dB'}

selected_attenuation = 'total' ## Change here to select another atenuation

file_name = attenuations_file[selected_attenuation]
att_col = att_columns[selected_attenuation]
header = 6

df_lavrion_375_1 = pd.read_csv(f"data/{folders_name[0]}/output/ascii/{file_name}",header=header)
df_lavrion_375_2 = pd.read_csv(f"data/{folders_name[1]}/output/ascii/{file_name}",header=header)

df_lavrion_375 = pd.concat([df_lavrion_375_1, df_lavrion_375_2], axis=0, ignore_index=True)
df_lavrion_375.name = "Lavrion 37.5GHz"

df_lavrion_75_1 = pd.read_csv(f"data/{folders_name[2]}/output/ascii/{file_name}",header=header)
df_lavrion_75_2 = pd.read_csv(f"data/{folders_name[3]}/output/ascii/{file_name}",header=header)

df_lavrion_75 = pd.concat([df_lavrion_75_1, df_lavrion_75_2], axis=0, ignore_index=True)
df_lavrion_75.name = "Lavrion 75GHz"

df_ljubljana_375_1 = pd.read_csv(f"data/{folders_name[4]}/output/ascii/{file_name}",header=header)
df_ljubljana_375_2 = pd.read_csv(f"data/{folders_name[5]}/output/ascii/{file_name}",header=header)

df_ljubljana_375 = pd.concat([df_ljubljana_375_1, df_ljubljana_375_2], axis=0, ignore_index=True)
df_ljubljana_375.name = "Ljubljana 37.5GHz"

df_ljubljana_75_1 = pd.read_csv(f"data/{folders_name[6]}/output/ascii/{file_name}",header=header)
df_ljubljana_75_2 = pd.read_csv(f"data/{folders_name[7]}/output/ascii/{file_name}",header=header)

df_ljubljana_75 = pd.concat([df_ljubljana_75_1, df_ljubljana_75_2], axis=0, ignore_index=True)
df_ljubljana_75.name = "Ljubljana 75GHz"

dfs = [df_lavrion_375, df_lavrion_75, df_ljubljana_375, df_ljubljana_75]



In [ ]:
import plotly.graph_objects as go

for data_df in dfs:
    unique_elev = np.unique(data_df["Elevation_angle_deg"])
    fig = go.Figure()
    for elev_angle in unique_elev:
        temp_df =  data_df[data_df["Elevation_angle_deg"] == elev_angle]
        fig.add_trace(go.Scatter(x=temp_df['Time_exceeded_%'], y=temp_df[att_col], name=f"Elevation {elev_angle}"))
        fig.add_trace(go.Scatter(mode='markers',marker_symbol="x",x=temp_df['Time_exceeded_%'], y=temp_df[att_col],name=f"Elevation {elev_angle}"))

    fig.update_xaxes(type="log")
    fig.update_layout(title=f"{data_df.name}, {file_name}", xaxis_title="Time exceeded  [%]log", yaxis_title=f"Attenuation {selected_attenuation} [dB]")
    fig.show()




## Attenuation with respect of elevation

In [ ]:
time_exceeded=data_df['Time_exceeded_%'].array[0]
print("Time exceeded: ", time_exceeded,"%")
for data_df in dfs:
    threshold_vs_elev = []

    for elev in stat.index.array[:-1]:
        threshold_vs_elev.append(data_df[data_df['Elevation_angle_deg']==elev][att_col][data_df['Time_exceeded_%']==time_exceeded].array[0]) # 
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=stat.index.array[:-1],y=threshold_vs_elev))
    fig.update_layout(title=f"{data_df.name}, {selected_attenuation} attenuation", xaxis_title="elevation [degrees]", yaxis_title=f" {selected_attenuation} Attenuation[dB]")    
    fig.show()

## Compute average attenuation

In [ ]:
ponderated_df = {}
interpolation_number =100
for data_df in dfs:
    ponderated_attenuation = 0
    new_percentage = np.geomspace(data_df[data_df['Elevation_angle_deg']==5]['Time_exceeded_%'].array[0],data_df[data_df['Elevation_angle_deg']==5]['Time_exceeded_%'].array[-1],interpolation_number)
    for elev in stat.index.array[:-1]:
        fcn = interpolate.interp1d(data_df[data_df['Elevation_angle_deg']==elev]['Time_exceeded_%'].array,data_df[data_df['Elevation_angle_deg']==elev][att_col].array)
        ponderated_attenuation += fcn(new_percentage)*stat['PDF'][elev]/100
        #ponderated_attenuation +=  data_df[data_df['Elevation_angle_deg']==elev][att_col].array*stat['PDF'][elev]/100 # /100 to transform % to decimal
    ponderated_df[data_df.name] = {'attenuation':ponderated_attenuation,'time_exceeded':new_percentage}
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=new_percentage,y=ponderated_attenuation))
    #fig.add_trace(go.Scatter(x=data_df['Time_exceeded_%'],y=ponderated_attenuation)) ## Without interpolation
    fig.update_xaxes(type="log")
    fig.update_layout(title=f"{data_df.name}, {selected_attenuation} attenuation", xaxis_title="Time exceeded  [%]log", yaxis_title=f" {selected_attenuation} Attenuation[dB]")    
    fig.add_vline(x=1, line_width=3, line_dash="dash", line_color="red")
    fig.add_vline(x=0.5, line_width=3, line_dash="dash", line_color="green")
    fig.add_vline(x=0.1, line_width=3, line_dash="dash", line_color="yellow")
    fig.add_vline(x=0.01, line_width=3, line_dash="dash", line_color="orange")
    fig.show()

## Transmitter EIRP calculation

EIRP = SNR - Gr + L + N

In [ ]:
from scipy import interpolate

def avg_distances(city_name):
    ## Average distance calculation
    pkl_name= f"./coordinates_{city_name}.pkl" 
    pkl_pdf_name= f"./stat_{city_name}.pkl" 
    coord_df = pd.DataFrame(pd.read_pickle(pkl_name))

    ind  = np.logical_and( 0 <= coord_df['EL'] , coord_df['EL'] < 90) # For all elevation that are in the range 0, 90 degrees
    distances = coord_df['DISTANCES'][ind]
    print("Distances statistics")
    print(coord_df['DISTANCES'][np.logical_and( 0 <= coord_df['EL'] , coord_df['EL'] < 90)].describe())

    return np.mean(distances) # Earth-satellite mean distance [meter]

## To EDIT
T = 500                                     # System temperature [Kelvin]
Gr = 51                                   # Receive antenna gain [dB]
city_name='lavrion'                       # City name used to compute the average distance betzeen BS and satellite
selected_bs_freq = 'Lavrion 75GHz'        # ponderated_df_keys = ['Lavrion 37.5GHz', 'Lavrion 75GHz', 'Ljubljana 37.5GHz', 'Ljubljana 75GHz'] used to get ponderated attenuation
fr = 75*10**9                               # frequency GHz


## Constants
k = 1.381e-23 # boltzman constant
B = 50        # Receiver bandwidth [Hz]
SNR = 10      # signal to noise ratio [dB]

N = 10*np.log10(k * T * B)  # Noise level

lamda = 3*10**8/fr # wavelength [meter]


r=avg_distances(city_name)


## Loss calculation
L_0 = 20*np.log10(4*np.pi*r*1000/lamda) # free space loss [dB]
L_atm = ponderated_df[selected_bs_freq]['attenuation'] # Atmospheric attenuation for every probability [dB]
L_misc = 6                              # Micellaneous loss [dB]


L = L_0+L_atm+L_misc                    # total path loss


eirp = SNR + N + L - Gr

time_exceeded = ponderated_df[selected_bs_freq]['time_exceeded']
fig = go.Figure()
fig.add_trace(go.Scatter(x=time_exceeded,y=eirp))
fig.update_layout(title=f"{selected_bs_freq}", xaxis_title="Time exceeded  [%]log", yaxis_title=f"EIRP [dB]")    
fig.update_xaxes(type="log")
fig.show()

